### Diese Notebook zeigt EVENTUELL ZUKÜNFTIG das Artefakt als prototypische Front-End-Anwendung

Zielt eventuell auf Präsentation ab, und deshalb noch NICHT FINAL!
Hier nur erste Versuche mit HTML-Layout. 



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
from PIL import Image, ImageTk
from faker import Faker
from transformers import pipeline

Generierung & Verifizierung als Gradio-Application

In [ ]:
def generate_review_html(stars, date, user, is_local_guide, text, user_image, review_images, label):
    full_stars = int(stars)
    half_star = (stars % 1) > 0
    empty_stars = 5 - full_stars - half_star
    full_star_html = '<span class="star full">&#9733;</span>'
    half_star_html = '<span class="star half">&#9733;</span>'
    empty_star_html = '<span class="star empty">&#9733;</span>'
    stars_html = (full_star_html * full_stars) + (half_star_html * half_star) + (empty_star_html * empty_stars)
    user_name = user if not is_local_guide else f"{user} (local guide)"
    
    html = f"""
    <style>
      .star.full {{
        color: #f1c40f;
      }}
      .star.half {{
        color: #f1c40f;
        position: relative;
      }}
      .star.half::before {{
        content: '\\2605';
        position: absolute;
        left: 0;
        color: #ddd;
      }}
      .star.empty {{
        color: #ddd;
      }}
    </style>
    <div class="review">
      <div class="user">
        <div class="user-info">
          <h4>{user_name}</h4>
          <p>Submitted on {date}</p>
          <p class="hidden">{label}</p>
        </div>
      </div>
      <div class="rating">
        {stars_html}
      </div>
      <div class="review-text">
        <p>{text}</p>
      </div>
      <div class="review-image">
        <img src={review_images} alt="Restaurant image">
      </div>
    </div>
    """
    return html

In [ ]:
import openai
fake = Faker()

In [214]:
def generate_deepfake(review_about, stars):
    date = fake.date_between(start_date='-1y', end_date='today')
    user = fake.name()
    local_guide = fake.boolean(chance_of_getting_true=4)
    new_prompt = f"A Google Maps review about a {review_about}. ###"

    with open("apikey_moritz.txt", "r") as f:
        api_key = f.read()
    print(api_key)
    openai.api_key = api_key
    fine_tune_list = openai.FineTune.list()
    fine_tuned_model = fine_tune_list['data'][0].fine_tuned_model
    
    print(fine_tuned_model)  
    answer = openai.Completion.create(
    model=fine_tuned_model,
    prompt=new_prompt,
    max_tokens=200, # Change amount of tokens for longer completion
    temperature=1,
    stop = "END"
    )
    text = "Das ist ein toller Tesbericht! Liebe es!"
    text = answer['choices'][0]['text']
    print(text)

    # Image
    sentiment = pipeline('sentiment-analysis')(text)[0]['label']
    print(f"{sentiment} Text"
          )
    if sentiment == "POSITIVE":
       sentiment = "great"
       stars = fake.random_int(min=4, max=5)
    elif sentiment == "NEUTRAL":
        sentiment = "average"
        stars = fake.random_int(min=3, max=4)
    elif sentiment == "NEGATIVE":
        sentiment = "okay"
        stars = fake.random_int(min=1, max=2)
    img_prompt =  f"A photo of a {sentiment} time at a {review_about},  taken by a photograph visitor. The shot is taken Shot taken by a iPhone 13 Pro, 26mm, f/2.8, ISO 100, hyper detailed, Super-Resolution, UHD, DTM, HDR, 8K --ar 4:3 --q 2, on Tripadvisor.com 2022"
    
    with open("apikey_valerie.txt", "r") as f:
        api_key = f.read()
    response = openai.Image.create(prompt=img_prompt, n=1, size="256x256")
    img_url = response["data"][0]["url"]

    html_review = generate_review_html(stars, date, user, local_guide, text, user_image="https://via.placeholder.com/50/%22%20alt=/%22Restaurant%20image/", review_images = img_url, label = "fake")
    return html_review

## V2 Variante: Mit Stars angeben, veraltetes Modell

In [ ]:
fake = Faker()

In [215]:
def generate_deepfake_ft0(review_about, stars, keywords):
    date = fake.date_between(start_date="-1y", end_date="today")
    user = fake.name()
    local_guide = fake.boolean(chance_of_getting_true=4)

    with open("apikey_openai.txt", "r") as f:
        api_key = f.read()

    openai.api_key = api_key
    fine_tune_list = openai.FineTune.list()
    fine_tuned_model = fine_tune_list["data"][0].fine_tuned_model
    print(fine_tuned_model)

    new_prompt = f"A {stars} stars Google Maps review about a {review_about} witth follwing keywords: {keywords} ###"
    answer = openai.Completion.create(
        model=fine_tuned_model,
        prompt=new_prompt,
        max_tokens=200,  # Change amount of tokens for longer completion
        temperature=1,
        stop="END",
    )
    text = "Das ist ein toller Tesbericht! Liebe es!"
    text = answer["choices"][0]["text"]
    print(text)

    if stars == 5:
        sentiment = "great"
    elif stars == 4:
        sentiment = "good"
    elif stars == 3:
        sentiment = "average"
    elif stars == 2:
        sentiment = "okay"
    elif stars == 1:
        sentiment = "okay"
    img_prompt = f"A photo of a {sentiment} time at a {review_about},  taken by a photograph visitor. The shot is taken Shot taken by a iPhone 13 Pro, 26mm, f/2.8, ISO 100, hyper detailed, Super-Resolution, UHD, DTM, HDR, 8K --ar 4:3 --q 2, on Tripadvisor.com 2022"
    with open("apikey_valerie.txt", "r") as f:
        api_key = f.read()
    response = openai.Image.create(prompt=img_prompt, n=1, size="256x256")
    img_url = response["data"][0]["url"]

    html_review = generate_review_html(
        stars,
        date,
        user,
        local_guide,
        text,
        user_image="https://via.placeholder.com/50/%22%20alt=/%22Restaurant%20image/",
        review_images=img_url,
        label="fake"
    )
    return html_review

## Fake-Oder-Echt-Quiz

In [ ]:
df = pd.read_csv("01_Data/data_cleaned_processed/base_for_feature_extraction.csv")
df.head(55)[["text", "categoryName"]]

In [213]:
import gradio as gr
import pandas as pd
import re

df = pd.read_csv("01_Data/data_cleaned_processed/base_for_feature_extraction.csv")
df = df.drop(df.index[16:26])
df = df.sample(frac=1, random_state=42)

## ist die Frage ob ich den Index zurücksezte und dann daran die beispiele raussuchen soll 
#df = df.reset_index(drop=True)

# keep_index = [46,53]
# df = df.iloc[keep_index]

current_review_index = 0
count_right_answers = 0

reviews = []
for index, row in df.iterrows():
    stars = row["stars"]
    date = row["publishedAtDate"]
    user = row["name"]
    is_local_guide = row["isLocalGuide"]
    text = row["text"]
    label = row["label"]
    user_image = ""
    category = row["categoryName"]
    title = row["title"]
    review_images = row["reviewImageUrls/0"]
    html_input = generate_review_html(stars, date, user, is_local_guide, text, user_image, review_images, label)
    reviews.append(html_input)

def generate_gradio_input (html_input, answer): 
    global current_review_index
    global count_right_answers

    # Auswertung der abgegebenen Antwort
    if "fake" in html_input:
        label  ="fake"
    else:
        label = "real"
    is_fake = label == "fake"

    if answer.lower() == "fake" and is_fake:
        feedback = "Korrekt! Die Review war fake."
        count_right_answers += 1
    elif answer.lower() == "real" and not is_fake:
        feedback = "Korrekt! Die Review war real."
        count_right_answers += 1
    else:
        feedback = "Leider nicht korrekt! Die Review war " + ("fake." if is_fake else "real.")

    current_review_index += 1
    infotext = feedback + f"\n {count_right_answers} von {current_review_index} richtig erkannt ({round(count_right_answers/current_review_index*100,2)} %)."
    infotext_html = gr.HTML(f"<h3> {infotext} </h3>")

    # Ausgabe der nächsten Bewertung
    print(f"Index: {current_review_index}")
    if current_review_index >= len(reviews):
         return None
    review_html = reviews[current_review_index]
    input_html = gr.HTML(review_html)
    answer_pos = gr.Radio(["real", "fake"], label="Ist diese Bewertung echt oder gefälscht?")
    kategorietext_html = f"<h2> Eine Google Maps Reviews aus der Kategorie: {df.iloc[current_review_index]['categoryName']} <hr></h2>"
    return review_html, infotext, kategorietext_html

with gr.Blocks() as demo:
    with gr.Tab ("Quiz: Fake oder Echt?"):
        with gr.Row():
            with gr.Column():
                category = df.iloc[current_review_index]["categoryName"]
                kurze_info = gr.HTML(f"<h3> Eine Google Maps Reviews aus der Kategorie: {df.iloc[current_review_index]['categoryName']} <hr></h3>")
                input_html = gr.HTML(reviews[3])
            with gr.Column(Label="Tipp und Auswertung"):
                answer_pos = gr.Radio(["fake", "real"], label="Ist diese Bewertung echt oder gefälscht?")
                submit_btn = gr.Button("Auswerten")
                infotext = gr.Textbox(label="Auflösung vorheriger Antwort:", readonly=True)
                submit_btn.click(generate_gradio_input, inputs=[input_html, answer_pos], outputs=[input_html, infotext, kurze_info])

    with gr.Tab ("Create Fake Review"):
        review_about = gr.Textbox(label="Review about")
        review_stars = gr.Slider(label="Stars", minimum=1, maximum=5, step=1)
        generate_btn = gr.Button("Multimodale Bewertung erstellen")
        html_output = gr.HTML(label="Review") 
        generate_btn.click(fn=generate_deepfake, inputs=[review_about, review_stars], outputs=[html_output])

    with gr.Tab ("Create Fake Review FT1"):
        review_about = gr.Textbox(label="Review about")
        review_stars = gr.Slider(label="Stars", minimum=1, maximum=5, step=1)
        review_keywords = gr.Textbox(label="Keywords")
        generate_btn = gr.Button("Generate Multi-Modal Review")
        html_output = gr.HTML(label="Review") 
        generate_btn.click(fn=generate_deepfake_ft0, inputs=[review_about, review_stars], outputs=[html_output])
            
demo.launch(share=True)

c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\deprecation.py:43: UserWarning: You have unused kwarg parameters in Column, please remove them: {'Label': 'Tipp und Auswertung'}
  warnings.warn(
c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\deprecation.py:43: UserWarning: You have unused kwarg parameters in Textbox, please remove them: {'readonly': True}
  warnings.warn(
c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\utils.py:805: UserWarning: Expected 3 arguments for function <function generate_deepfake_ft0 at 0x0000023A9D4A83A0>, received 2.
  warnings.warn(
c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\utils.py:809: UserWarning: Expected at least 3 arguments for function <function generate_deepfake_ft0 at 0x0000023A9D4A83A0>, received 2.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7900
Running on public URL: https://ddb4020a-8bab-46e4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Index: 1
Index: 2
Index: 3
Index: 4
Index: 5
Index: 6
Index: 7
Index: 8
Index: 9
Index: 10
Index: 11
Index: 12
Index: 13
Index: 14
Index: 15
Index: 16


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


 Amazing food, very cozy.  Good background music.  Don't forget to try the desserts and Creme brûlée.  The sausage and mashed potatoes are from another planet. 
POSITIVE Text


Traceback (most recent call last):
  File "c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\routes.py", line 337, in run_predict
    output = await app.get_blocks().process_api(
  File "c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\blocks.py", line 1015, in process_api
    result = await self.call_function(
  File "c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\blocks.py", line 833, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\anyio\to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\anyio\_backends\_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\anyio\_backends\_as

 This restaurant is located in a strip mall, fairly easy to find. The parking area is narrow and faces the building next door—but there is parking available. The interior contains several tables of various sizes (for 2 to 6 people) for dining. The layout is pretty open, and you can see almost all of the dining area.
Estel's is open for breakfast, lunch and dinner. Their menu includes a variety of German foods (sausage, jagerschnitzel , leberwurst, breze, etc.) If you look carefully, you'll see their marque says 'family orientated' and they should receive points for that immediately! # 1: The quality of the food is consistently good. # 2: The price is reasonable. On entry into the dining area, they give you a tray with a linen-like cloth on it. The atmosphere is pleasant, filled with people of all ages. Service is good, and generally your food is brought to you


Traceback (most recent call last):
  File "c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\routes.py", line 337, in run_predict
    output = await app.get_blocks().process_api(
  File "c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\blocks.py", line 1015, in process_api
    result = await self.call_function(
  File "c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\blocks.py", line 833, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\anyio\to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\anyio\_backends\_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\anyio\_backends\_as

Bisher: Mit Layout rumgespielt. Noch keine **Funktionalität der Deepfake-Modelle** intergiert!

In [211]:
with gr.Blocks() as demo:
    with gr.Tab ("Create Fake Review"):
        review_about = gr.Textbox(label="Review about")
        review_stars = gr.Slider(label="Stars", minimum=1, maximum=5, step=1)
        generate_btn = gr.Button("Generate Multi-Modal Review")
        html_output = gr.HTML(label="Review") 
        generate_btn.click(fn=generate_deepfake, inputs=[review_about, review_stars], outputs=[html_output])
    
    with gr.Tab ("Create Fake Review FT1"):
        review_about = gr.Textbox(label="Review about")
        review_stars = gr.Slider(label="Stars", minimum=1, maximum=5, step=1)
        review_keywords = gr.Textbox(label="Keywords")
        generate_btn = gr.Button("Generate Multi-Modal Review")
        html_output = gr.HTML(label="Review") 
        generate_btn.click(fn=generate_deepfake_ft0, inputs=[review_about, review_stars], outputs=[html_output])

    with gr.Tab ("Quiz: Fake oder Echt?"):
        df = pd.read_csv("01_Data/data_cleaned_processed/base_for_feature_extraction.csv")
        print(df.shape)
        print(df.columns)
        random_review = df.sample(1)
        stars = random_review["stars"].values[0]
        date = random_review["publishedAtDate"].values[0]
        user = random_review["name"].values[0]
        is_local_guide = random_review["isLocalGuide"].values[0]
        text = random_review["text"].values[0]
        user_image = ""
        label =  random_review["label"].values[0]
        review_images = random_review["reviewImageUrls/0"].values[0]

        html_input = generate_review_html(stars, date, user, is_local_guide, text, user_image, review_images)
        print(html_input)
        html_output = gr.HTML(html_input, label="Review")        
        check_btn = gr.Radio(["Fake", "Real"], label="Fake or Real?")
        check_btn.click( , inputs=[review_about, review_stars], outputs=[html_output])

demo.launch(debug=True)

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│         check_btn.click( , inputs=[review_about, review_stars], outputs=[html_output])           │
│                          ▲                                                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax